[시도해 볼 것]
- 1) 3개 -> 4번째, 5번째 각각.
- 2) 3개 -> 4번째 예측 -> 묶어서 5번째.
- 3) (0, 3) -> 4, (1, 4) -> 5.

In [1]:
# 모듈 불러오기
from collections import Counter

import nltk
nltk.download('punkt')
from nltk import word_tokenize

from tensorflow.keras.layers import Input,Dense,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

import numpy as np
import string
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
# 경로 설정
root_path = "/content/drive/My Drive/멀티캠퍼스/[혁신성장] 인공지능 자연어처리 기반/[강의]/조성현 강사님"
data_path = f"{root_path}/dataset"

# _1_. 데이터

* 데이터 생김새

```
Alice\'s Adventures in Wonderland\n\n                ALICE\'S ADVENTURES IN WONDERLAND\n\n                          Lewis Carroll\n\n               THE MILLENNIUM FULCRUM EDITION 3.0\n\n\n\n\n                            CHAPTER I\n\n                      Down the Rabbit-Hole\n\n\n  Alice was beginning to get very tired of sitting by her sister\non the bank, and of having nothing to do:  once or twice she had\npeeped into the book her sister was reading, but it had no\npictures or conversations in it, `and what is the use of a book,\'\nthought Alice `without pictures or conversation?\'\n\n  So she was considering in her own mind (as well as she could,\nfor the hot day made her feel very sleepy and stupid),
```



In [ ]:
# 데이터 로드
with open(f"{data_path}/alice_in_wonderland.txt", "r") as f:
    content = f.read()

# _2_. 전처리

## 1) 구둣점 제거

* 구둣점 제거

    - `[" " if char in string.punctuation else char for char in content]` : 한 글자씩
    ```
    'r',
 'a',
 'n',
 ' ',
 'c',
 'l',
 'o'
 ```

    - `"".join([" " if char in string.punctuation else char for char in content])` : 띄어쓰기 되어 있음
    ```
    Alice s Adventures in Wonderland\n\n                ALICE S ADVENTURES IN WONDERLAND\n\n                          Lewis Carroll\n\n               THE MILLENNIUM FULCRUM EDITION 3 0\n\n\n\n\n                            CHAPTER I\n\n                      Down the Rabbit Hole\n\n\n  Alice was beginning to get very tired of sitting by her sister\non the bank  and of having nothing to do   once or twice she had\npeeped into the book her sister was reading  but it had n
    ```
    - `"".join([" " if char in string.punctuation else char for char in content].split()` : 단어별로 구분
    - 최종
    ```
    Alice s Adventures in Wonderland ALICE S ADVENTURES IN WONDERLAND Lewis Carroll THE MILLENNIUM FULCRUM EDITION 3 0 CHAPTER I Down the Rabbit Hole Alice was beginning to get very tired of sitting by her sister on the bank and of having nothin
    ```

## 2) 토큰화

## 3) 소문자화

## 4) 철자 2개 이상


In [ ]:
# 구둣점 제거
text = " ".join("".join([" " if char in string.punctuation else char for char in content]).split())
text

'Alice s Adventures in Wonderland ALICE S ADVENTURES IN WONDERLAND Lewis Carroll THE MILLENNIUM FULCRUM EDITION 3 0 CHAPTER I Down the Rabbit Hole Alice was beginning to get very tired of sitting by her sister on the bank and of having nothing to do once or twice she had peeped into the book her sister was reading but it had no pictures or conversations in it and what is the use of a book thought Alice without pictures or conversation So she was considering in her own mind as well as she could for the hot day made her feel very sleepy and stupid whether the pleasure of making a daisy chain would be worth the trouble of getting up and picking the daisies when suddenly a White Rabbit with pink eyes ran close by her There was nothing so VERY remarkable in that nor did Alice think it so VERY much out of the way to hear the Rabbit say to itself Oh dear Oh dear I shall be late when she thought it over afterwards it occurred to her that she ought to have wondered at this but at the time it al

In [ ]:
# 토큰화
tokens = word_tokenize(text)
tokens = [word.lower() for word in tokens if len(word) >= 2]
tokens

['alice',
 'adventures',
 'in',
 'wonderland',
 'alice',
 'adventures',
 'in',
 'wonderland',
 'lewis',
 'carroll',
 'the',
 'millennium',
 'fulcrum',
 'edition',
 'chapter',
 'down',
 'the',
 'rabbit',
 'hole',
 'alice',
 'was',
 'beginning',
 'to',
 'get',
 'very',
 'tired',
 'of',
 'sitting',
 'by',
 'her',
 'sister',
 'on',
 'the',
 'bank',
 'and',
 'of',
 'having',
 'nothing',
 'to',
 'do',
 'once',
 'or',
 'twice',
 'she',
 'had',
 'peeped',
 'into',
 'the',
 'book',
 'her',
 'sister',
 'was',
 'reading',
 'but',
 'it',
 'had',
 'no',
 'pictures',
 'or',
 'conversations',
 'in',
 'it',
 'and',
 'what',
 'is',
 'the',
 'use',
 'of',
 'book',
 'thought',
 'alice',
 'without',
 'pictures',
 'or',
 'conversation',
 'so',
 'she',
 'was',
 'considering',
 'in',
 'her',
 'own',
 'mind',
 'as',
 'well',
 'as',
 'she',
 'could',
 'for',
 'the',
 'hot',
 'day',
 'made',
 'her',
 'feel',
 'very',
 'sleepy',
 'and',
 'stupid',
 'whether',
 'the',
 'pleasure',
 'of',
 'making',
 'daisy',
 'ch

# _3_. N그램

In [ ]:
N = int(input('N 개수 설정: '))

quads = list(nltk.ngrams(tokens, N))
quads

N 개수 설정: 5


[('alice', 'adventures', 'in', 'wonderland', 'alice'),
 ('adventures', 'in', 'wonderland', 'alice', 'adventures'),
 ('in', 'wonderland', 'alice', 'adventures', 'in'),
 ('wonderland', 'alice', 'adventures', 'in', 'wonderland'),
 ('alice', 'adventures', 'in', 'wonderland', 'lewis'),
 ('adventures', 'in', 'wonderland', 'lewis', 'carroll'),
 ('in', 'wonderland', 'lewis', 'carroll', 'the'),
 ('wonderland', 'lewis', 'carroll', 'the', 'millennium'),
 ('lewis', 'carroll', 'the', 'millennium', 'fulcrum'),
 ('carroll', 'the', 'millennium', 'fulcrum', 'edition'),
 ('the', 'millennium', 'fulcrum', 'edition', 'chapter'),
 ('millennium', 'fulcrum', 'edition', 'chapter', 'down'),
 ('fulcrum', 'edition', 'chapter', 'down', 'the'),
 ('edition', 'chapter', 'down', 'the', 'rabbit'),
 ('chapter', 'down', 'the', 'rabbit', 'hole'),
 ('down', 'the', 'rabbit', 'hole', 'alice'),
 ('the', 'rabbit', 'hole', 'alice', 'was'),
 ('rabbit', 'hole', 'alice', 'was', 'beginning'),
 ('hole', 'alice', 'was', 'beginning', 

In [ ]:
# N 그램 학습 데이터 형태로 변환
new_text = []
for q in quads:
    temp = " ".join(q)
    new_text.append(temp)

new_text

['alice adventures in wonderland alice',
 'adventures in wonderland alice adventures',
 'in wonderland alice adventures in',
 'wonderland alice adventures in wonderland',
 'alice adventures in wonderland lewis',
 'adventures in wonderland lewis carroll',
 'in wonderland lewis carroll the',
 'wonderland lewis carroll the millennium',
 'lewis carroll the millennium fulcrum',
 'carroll the millennium fulcrum edition',
 'the millennium fulcrum edition chapter',
 'millennium fulcrum edition chapter down',
 'fulcrum edition chapter down the',
 'edition chapter down the rabbit',
 'chapter down the rabbit hole',
 'down the rabbit hole alice',
 'the rabbit hole alice was',
 'rabbit hole alice was beginning',
 'hole alice was beginning to',
 'alice was beginning to get',
 'was beginning to get very',
 'beginning to get very tired',
 'to get very tired of',
 'get very tired of sitting',
 'very tired of sitting by',
 'tired of sitting by her',
 'of sitting by her sister',
 'sitting by her sister o

# _4_. 벡터화: CountVectorizer 이용

- count vectorizer로 벡터화하면 가각ㄱ의 단어에서 위치가 어디있는지 나온다.
- X matrix에는 3개 있어야 하고, y matrix에는 2개 있어야 한다.

In [ ]:
# countvectorize

vectorizer = CountVectorizer()

X_trigram_1 = []
X_trigram_2 = []
y_trigram_1 = []
y_trigram_2 = []

for text in new_text:
    x_str_1 = " ".join(text.split()[0:N-2])
    x_str_2 = " ".join(text.split()[1:N-1])
    y_str_1 = text.split()[N-2] 
    y_str_2 = text.split()[N-1]

    X_trigram_1.append(x_str_1)
    X_trigram_2.append(x_str_2)
    y_trigram_1.append(y_str_1)
    y_trigram_2.append(y_str_2)

In [ ]:
# countvertorizer 이용 벡터화
X_trigram_1_mat = vectorizer.fit_transform(X_trigram_1).todense()
X_trigram_2_mat = vectorizer.fit_transform(X_trigram_2).todense() 
y_trigram_1_mat = vectorizer.fit_transform(y_trigram_1).todense()
y_trigram_2_mat = vectorizer.fit_transform(y_trigram_2).todense()

# _5_. 어휘집
- countvectorizer의 vocabulary_ 메소드 쓰면 워드 투 인덱스 어휘집이 나온다.



In [ ]:
# 어휘집
word2idx = vectorizer.vocabulary_
idx2word = {v:k for k, v in word2idx.items()} 

# _6_. 모델 구성

In [ ]:
# 데이터 준비
X_1 = np.array(X_trigram_1_mat)
X_2 = np.array(X_trigram_2_mat)
y_1 = np.array(y_trigram_1_mat)
y_2 = np.array(y_trigram_2_mat)

# train, test 분리
X_train_1, X_test_1, y_train_1, y_test_1, X_train_1_tg, X_test_1_tg = train_test_split(X_1, y_1, X_trigram_1,
                                                                                        test_size=0.3, 
                                                                                        random_state=42)
print(f"Train: {X_train_1.shape}, {y_train_1.shape}")
print(f"Test: {X_test_1.shape}, {y_test_1.shape}")
print(f"Train Trigram: {len(X_train_1_tg)}, {len(X_test_1_tg)}")

X_train_2, X_test_2, y_train_2, y_test_2, X_train_2_tg, X_test_2_tg = train_test_split(X_2, y_2, X_trigram_2,
                                                                                        test_size=0.3, 
                                                                                        random_state=42)
print(f"Train: {X_train_2.shape}, {y_train_2.shape}")
print(f"Test: {X_test_2.shape}, {y_test_2.shape}")
print(f"Train Trigram: {len(X_train_2_tg)}, {len(X_test_2_tg)}")

Train: (17945, 2559), (17945, 2559)
Test: (7692, 2559), (7692, 2559)
Train Trigram: 17945, 7692
Train: (17945, 2559), (17945, 2559)
Test: (7692, 2559), (7692, 2559)
Train Trigram: 17945, 7692


In [ ]:
# 모델 파라미터 설정
EPOCHS = int(input('학습 에폭 수 설정: '))
BATCH = int(input('배치 사이즈 설정: '))

학습 에폭 수 설정: 300
배치 사이즈 설정: 100


In [ ]:
# FFN 네트워크 구성
X_input = Input(shape=(X_train_1.shape[1], ), name='input')
X_dense_1 = Dense(1000, activation='relu', name='1st_hidden')(X_input)
# X_dense_1 = Dropout(0.5, name='1st_dout')(X_dense_1)
X_dense_2 = Dense(800, activation='relu', name='2nd_hidden')(X_dense_1)
X_dense_3 = Dense(1000, activation='relu', name='3rd_hidden')(X_dense_2)
# X_dense_3 = Dropout(0.5, name='2nd_dout')(X_dense_3)
y_output = Dense(y_train_1.shape[1], activation='softmax', name='output')(X_dense_3)

In [ ]:
# 모델 구성
model1 = Model(X_input, y_output)
model1.compile(optimizer=Adam(lr=0.025), loss='categorical_crossentropy')
print("======= 모델1 전체 구조 ======")
print(model1.summary())
print()
model2 = Model(X_input, y_output)
model2.compile(optimizer=Adam(lr=0.025), loss='categorical_crossentropy')
print("======= 모델2 전체 구조 ======")
print(model2.summary())

======= 모델1 전체 구조 ======
Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 2559)]            0         
_________________________________________________________________
1st_hidden (Dense)           (None, 1000)              2560000   
_________________________________________________________________
2nd_hidden (Dense)           (None, 800)               800800    
_________________________________________________________________
3rd_hidden (Dense)           (None, 1000)              801000    
_________________________________________________________________
output (Dense)               (None, 2559)              2561559   
Total params: 6,723,359
Trainable params: 6,723,359
Non-trainable params: 0
_________________________________________________________________
None

======= 모델2 전체 구조 ======
Model: "model_9"
__________________________________________

# _7_. 학습

In [ ]:
# 모델1 훈련
# es = EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)
hist1 = model1.fit(X_train_1, y_train_1,
                   batch_size=BATCH,
                   epochs=EPOCHS,
                   validation_split=0.2)
print()
hist2 = model2.fit(X_train_2, y_train_2,
                   batch_size=BATCH,
                   epochs=EPOCHS,
                   validation_split=0.2)

Epoch 1/300
144/144 [==============================] - 1s 6ms/step - loss: 6.4534 - val_loss: 6.2392
Epoch 2/300
144/144 [==============================] - 1s 6ms/step - loss: 6.0920 - val_loss: 6.2303
Epoch 3/300
144/144 [==============================] - 1s 6ms/step - loss: 6.0338 - val_loss: 6.2667
Epoch 4/300
144/144 [==============================] - 1s 5ms/step - loss: 6.0221 - val_loss: 6.2729
Epoch 5/300
144/144 [==============================] - 1s 6ms/step - loss: 6.0223 - val_loss: 6.2962
Epoch 6/300
144/144 [==============================] - 1s 6ms/step - loss: 6.0181 - val_loss: 6.2883
Epoch 7/300
144/144 [==============================] - 1s 6ms/step - loss: 6.0152 - val_loss: 6.3716
Epoch 8/300
144/144 [==============================] - 1s 5ms/step - loss: 6.0168 - val_loss: 6.3087
Epoch 9/300
144/144 [==============================] - 1s 6ms/step - loss: 6.0106 - val_loss: 6.3836
Epoch 10/300
144/144 [==============================] - 1s 6ms/step - loss: 6.0108 - val_lo

# _8_. 결과 확인


In [ ]:
# # loss 시각화
# plt.plot(hist1.history['loss'], label='Train Loss')
# plt.title('Loss Trajectory')
# plt.legend()
# plt.show()

In [ ]:
# 모델 예측 값 확인
y_pred_1 = model1.predict(X_test_1)
y_pred_2 = model2.predict(X_test_2)

# 랜덤하게 예측된 라벨과 정답 확인
print ("이전 Trigram", " |Actual ", " |Predicted ","\n")

NUM_DISPLAY = int(input('확인할 라벨 수 설정: '))
# for i in random.sample(range(len(X_test_1_tg)), NUM_DISPLAY):

print()
for i in range(NUM_DISPLAY):
    print(i, X_test_1_tg[i], "|", idx2word[np.argmax(y_test_1[i])], idx2word[np.argmax(y_test_2[i])], "|", idx2word[np.argmax(y_pred_1[i])], idx2word[np.argmax(y_pred_2[i])])
    # print(i, X_test_2_tg[i], "|", idx2word[np.argmax(y_test_2[i])], "|", idx2word[np.argmax(y_pred_2[i])])

# ============ 테스트 ============

In [ ]:
# 모델 예측 값 확인
y_pred_1 = model1.predict(X_test_1)
y_pred_2 = model2.predict(X_test_2)

# 랜덤하게 예측된 라벨과 정답 확인
print ("이전 Trigram", " |Actual ", " |Predicted ","\n")

NUM_DISPLAY = int(input('확인할 라벨 수 설정: '))
# for i in random.sample(range(len(X_test_1_tg)), NUM_DISPLAY):

print()
for i in range(NUM_DISPLAY):
    print(i, X_test_1_tg[i], "|", idx2word[np.argmax(y_test_1[i])], "|", idx2word[np.argmax(y_pred_1[i])])
    print(i, X_test_2_tg[i], "|", idx2word[np.argmax(y_test_2[i])], "|", idx2word[np.argmax(y_pred_2[i])])

이전 Trigram  |Actual   |Predicted  

확인할 라벨 수 설정: 15
0 butter in the | other | march
0 in the other | beg | the
1 lap of her | sister | as
1 of her sister | who | skirt
2 the little dears | came | golden
2 little dears came | jumping | and
3 hadn quite finished | my | with
3 quite finished my | tea | with
4 if it began | ordering | ordering
4 it began ordering | people | people
5 re called lessons | the | after
5 called lessons the | gryphon | out
6 be in before | the | some
6 in before the | trial | wherever
7 and be turned | out | out
7 be turned out | of | of
8 then all the | party | things
8 all the party | were | at
9 hatter and in | that | that
9 and in that | direction | about
10 and she kept | on | felt
10 she kept on | puzzling | the
11 old turtle we | used | used
11 turtle we used | to | to
12 watch out of | its | its
12 out of its | waistcoat | mouth
13 every word you | fellows | ve
13 word you fellows | were | were
14 time there were | three | three
14 there were three | lit

In [ ]:
content

'Alice\'s Adventures in Wonderland\n\n                ALICE\'S ADVENTURES IN WONDERLAND\n\n                          Lewis Carroll\n\n               THE MILLENNIUM FULCRUM EDITION 3.0\n\n\n\n\n                            CHAPTER I\n\n                      Down the Rabbit-Hole\n\n\n  Alice was beginning to get very tired of sitting by her sister\non the bank, and of having nothing to do:  once or twice she had\npeeped into the book her sister was reading, but it had no\npictures or conversations in it, `and what is the use of a book,\'\nthought Alice `without pictures or conversation?\'\n\n  So she was considering in her own mind (as well as she could,\nfor the hot day made her feel very sleepy and stupid), whether\nthe pleasure of making a daisy-chain would be worth the trouble\nof getting up and picking the daisies, when suddenly a White\nRabbit with pink eyes ran close by her.\n\n  There was nothing so VERY remarkable in that; nor did Alice\nthink it so VERY much out of the way to he

In [ ]:
for i in range(5):
    print(X_trigram_1[i], " ", y_trigram_1[i])
    print(X_trigram_2[i], " ", y_trigram_2[i])
    print()

alice adventures in   wonderland
adventures in wonderland   alice

adventures in wonderland   alice
in wonderland alice   adventures

in wonderland alice   adventures
wonderland alice adventures   in

wonderland alice adventures   in
alice adventures in   wonderland

alice adventures in   wonderland
adventures in wonderland   lewis



In [ ]:
print(vectorizer.fit_transform(X_trigram)) # 각각 위치 어디에 있는지 나타냄.

  (0, 51)	1
  (0, 27)	1
  (0, 1088)	1
  (1, 27)	1
  (1, 1088)	1
  (1, 2510)	1
  (2, 51)	1
  (2, 1088)	1
  (2, 2510)	1
  (3, 51)	1
  (3, 27)	1
  (3, 2510)	1
  (4, 51)	1
  (4, 27)	1
  (4, 1088)	1
  (5, 27)	1
  (5, 1088)	1
  (5, 2510)	1
  (6, 1088)	1
  (6, 2510)	1
  (6, 1242)	1
  (7, 2510)	1
  (7, 1242)	1
  (7, 303)	1
  (8, 1242)	1
  :	:
  (25628, 1775)	1
  (25629, 1010)	1
  (25629, 1526)	1
  (25629, 1775)	1
  (25630, 1010)	1
  (25630, 1526)	1
  (25630, 347)	1
  (25631, 1526)	1
  (25631, 347)	1
  (25631, 1245)	1
  (25632, 70)	1
  (25632, 347)	1
  (25632, 1245)	1
  (25633, 2205)	1
  (25633, 70)	1
  (25633, 1245)	1
  (25634, 2205)	1
  (25634, 70)	1
  (25634, 970)	1
  (25635, 2205)	1
  (25635, 2146)	1
  (25635, 970)	1
  (25636, 507)	1
  (25636, 2146)	1
  (25636, 970)	1


In [ ]:
print(vectorizer.fit_transform(y_trigram)) # 각각 위치 어디에 있는지 나타냄.

  (0, 1088)	1
  (1, 2510)	1
  (2, 51)	1
  (3, 27)	1
  (4, 1088)	1
  (5, 2510)	1
  (6, 1242)	1
  (7, 303)	1
  (8, 2205)	1
  (9, 1363)	1
  (10, 869)	1
  (11, 641)	1
  (12, 334)	1
  (13, 591)	1
  (14, 2205)	1
  (15, 1719)	1
  (16, 1034)	1
  (17, 51)	1
  (18, 2430)	1
  (19, 183)	1
  (20, 2261)	1
  (21, 895)	1
  (22, 2398)	1
  (23, 2258)	1
  (24, 1478)	1
  :	:
  (25614, 2498)	1
  (25615, 53)	1
  (25616, 2206)	1
  (25617, 1975)	1
  (25618, 2053)	1
  (25619, 70)	1
  (25620, 784)	1
  (25621, 1624)	1
  (25622, 1088)	1
  (25623, 53)	1
  (25624, 2206)	1
  (25625, 1975)	1
  (25626, 1138)	1
  (25627, 1775)	1
  (25628, 1010)	1
  (25629, 1526)	1
  (25630, 347)	1
  (25631, 1245)	1
  (25632, 70)	1
  (25633, 2205)	1
  (25634, 970)	1
  (25635, 2146)	1
  (25636, 507)	1
  (25637, 2205)	1
  (25638, 664)	1


In [ ]:
np.argmax(np.array(vectorizer.fit_transform(X_trigram).todense())[0]) # 27번째 있는 단어

27

In [ ]:
np.argmax(np.array(vectorizer.fit_transform(X_trigram).todense())[1])

27

In [ ]:
np.argmax(np.array(vectorizer.fit_transform(X_trigram).todense())[2])

1088

In [ ]:
# 모델 예측 값 확인
y_pred_1 = model1.predict(X_test_1)
y_pred_2 = model2.predict(X_test_2)

# 랜덤하게 예측된 라벨과 정답 확인
print ("이전 Trigram", " |Actual ", " |Predicted ","\n")

NUM_DISPLAY = int(input('확인할 라벨 수 설정: '))
# for i in random.sample(range(len(X_test_1_tg)), NUM_DISPLAY):

print()
for i in range(NUM_DISPLAY):
    print(i, X_test_1_tg[i], "|", idx2word[np.argmax(y_test_1[i])], "|", idx2word[np.argmax(y_pred_1[i])])
    print(i, X_test_2_tg[i], "|", idx2word[np.argmax(y_test_2[i])], "|", idx2word[np.argmax(y_pred_2[i])])